In [1]:
!qiime --version

# 2019 7

q2cli version 2019.7.0
Run `qiime info` for more version details.


In [1]:
from metaBarTools import metaBar_PreX
import os

In [3]:
metaBar = metaBar_PreX()

> manifest for ITS was generated when working on 16s

> ITS working path was generated when working on ITS

In [2]:
path_ITS = "/media/swaggyp1985/HDD4T/OSU_Projects_2017-2018/running_project/2019/soil_project/micocosm/Microcosm/Analysis/updated_mcm/ITS2_result"

In [5]:
man_its = os.path.abspath("./manifest_mcm/ITS86@ITS4_manifest.csv")

In [3]:
metadatapath = os.path.abspath("./metadata/metadata_mcm_its_official.v2.tsv")

dive into analysis

In [4]:
os.chdir(path_ITS)
os.getcwd()

'/media/swaggyp1985/HDD4T/OSU_Projects_2017-2018/running_project/2019/soil_project/micocosm/Microcosm/Analysis/updated_mcm/ITS2_result'

In [8]:
!qiime tools import \
--type 'SampleData[PairedEndSequencesWithQuality]' \
--input-path $man_its \
--output-path mcm_its_seq.qza \
--input-format PairedEndFastqManifestPhred33

Imported /media/swaggyp1985/HDD4T/OSU_Projects_2017-2018/running_project/2019/soil_project/micocosm/Microcosm/manifest_mcm/ITS86@ITS4_manifest.csv as PairedEndFastqManifestPhred33 to mcm_its_seq.qza


In [9]:
!qiime demux summarize \
--i-data mcm_its_seq.qza \
--o-visualization mcm_its_seq.qzv

Saved Visualization to: mcm_its_seq.qzv


In [11]:
mcm_ITS_F = len("GTGAATCATCGAATCTTTGAA")
mcm_ITS_R = len("TCCTCCGCTTATTGATATGC")

In [10]:
if not os.path.exists("dada2_stats"):
    os.makedirs("dada2_stats")
if not os.path.exists("feature-tables"):
    os.makedirs("feature-tables")

In [12]:
#292
#227
#------
# 288
# 222
!qiime dada2 denoise-paired \
--i-demultiplexed-seqs mcm_its_seq.qza \
--output-dir dada2 \
--o-table feature-tables/table_mcm_its \
--o-representative-sequences rep_mcm_its \
--p-trim-left-f $mcm_ITS_F \
--p-trim-left-r $mcm_ITS_R \
--p-trunc-len-f 288 \
--p-trunc-len-r 222 \
--o-denoising-stats dada2_stats/dada2_stats_mcm_its.qza \
--p-n-threads 12

Saved FeatureTable[Frequency] to: feature-tables/table_mcm_its.qza
Saved FeatureData[Sequence] to: rep_mcm_its.qza
Saved SampleData[DADA2Stats] to: dada2_stats/dada2_stats_mcm_its.qza


In [13]:
!qiime metadata tabulate \
--m-input-file dada2_stats/dada2_stats_mcm_its.qza \
--o-visualization dada2_stats/stats-dada2.qzv

Saved Visualization to: dada2_stats/stats-dada2.qzv


In [ ]:
# adding meta data and cout table
!qiime feature-table summarize \
--i-table feature-tables/table_mcm_its.qza \
--o-visualization feature-tables/table_mcm_its.qzv \
--m-sample-metadata-file $metadatapath

## make trees

In [7]:
!qiime tools export \
--input-path rep_mcm_its.qza \
--output-path rep_mcm_its_seq

Exported rep_mcm_its.qza as DNASequencesDirectoryFormat to directory rep_mcm_its_seq


In [14]:
!qiime alignment mafft \
--i-sequences rep_mcm_its.qza \
--o-alignment aligned_rep_seqs.qza

Saved FeatureData[AlignedSequence] to: aligned_rep_seqs.qza


In [15]:
!qiime alignment mask \
--i-alignment aligned_rep_seqs.qza \
--o-masked-alignment masked_aligned_rep_seqs.qza

Saved FeatureData[AlignedSequence] to: masked_aligned_rep_seqs.qza


In [16]:
!qiime phylogeny fasttree \
--i-alignment masked_aligned_rep_seqs.qza \
--o-tree unrooted_tree.qza

Saved Phylogeny[Unrooted] to: unrooted_tree.qza


In [17]:
# root the tree
!qiime phylogeny midpoint-root \
--i-tree unrooted_tree.qza \
--o-rooted-tree rooted_tree.qza

Saved Phylogeny[Rooted] to: rooted_tree.qza


## Taxonomy

In [ ]:
db_seq_99_path = r"/media/swaggyp1985/HDD4T/OSU_Projects_2017-2018/Reads_metaData/Database/UNITE_INSD_ITS/UNITE_release/UNITE_UPDATE_2018-11-18/version2/sh_refs_qiime_ver8_99_s_02.02.2019.fasta"
db_tax_99_path = r"/media/swaggyp1985/HDD4T/OSU_Projects_2017-2018/Reads_metaData/Database/UNITE_INSD_ITS/UNITE_release/UNITE_UPDATE_2018-11-18/version2/sh_taxonomy_qiime_ver8_99_s_02.02.2019.txt"

In [18]:
if not os.path.exists("classifier"):
    os.makedirs("classifier")

In [ ]:
!qiime tools import \
--type "FeatureData[Sequence]" \
--input-path $db_seq_99_path \
--output-path ./classifier/UNITE_2019_99.qza

In [ ]:
!qiime tools import \
--type 'FeatureData[Taxonomy]' \
--input-format HeaderlessTSVTaxonomyFormat \
--input-path $db_tax_99_path\
--output-path ./classifier/UNITE_tax_2019_99.qza

In [ ]:
!qiime feature-classifier fit-classifier-naive-bayes \
--i-reference-reads ./classifier/UNITE_2019_99.qza \
--i-reference-taxonomy ./classifier/UNITE_tax_2019_99.qza \
--o-classifier ./classifier/classifier_UNITE_2019_99.qza

In [19]:
if not os.path.exists("taxonomy"):
    os.makedirs("taxonomy")

In [20]:
!qiime feature-classifier classify-sklearn \
--i-reads rep_mcm_its.qza \
--i-classifier ./classifier/classifier_UNITE_2019_99.qza \
--o-classification ./taxonomy/taxonomy_mcm_its.qza

Saved FeatureData[Taxonomy] to: ./taxonomy/taxonomy_mcm_its.qza


In [27]:
!qiime metadata tabulate \
--m-input-file ./taxonomy/taxonomy_mcm_its.qza \
--o-visualization ./taxonomy/taxonomy_mcm_its.qzv

Saved Visualization to: ./taxonomy/taxonomy_mcm_its.qzv


In [28]:
!qiime taxa barplot \
--i-table feature-tables/table_mcm_its.qza \
--i-taxonomy ./taxonomy/taxonomy_mcm_its.qza \
--m-metadata-file $metadatapath \
--o-visualization ./taxonomy/barplot_tax_mcm_its.qzv

Saved Visualization to: ./taxonomy/barplot_tax_mcm_its.qzv


### export for decontam

In [21]:
# !qiime tools export \
# --input-path feature-tables/table_mcm_its.qza \
# --output-path exported

Exported feature-tables/table_mcm_its.qza as BIOMV210DirFmt to directory exported


In [22]:
# !qiime tools export \
# --input-path taxonomy/taxonomy_mcm_its.qza \
# --output-path exported/

Exported taxonomy/taxonomy_mcm_its.qza as TSVTaxonomyDirectoryFormat to directory exported/


In [23]:
# !cp exported/taxonomy.tsv exported/biom-taxonomy.tsv

In [24]:
# # change header
# !sed -i '1 s/Feature ID/#OTUID/g; s/Taxon/taxonomy/g; s/Confidence/confidence/g' exported/biom-taxonomy.tsv

In [25]:
# !biom add-metadata \
# -i exported/feature-table.biom \
# -o exported/feature-table-tax.biom \
# --observation-metadata-fp exported/biom-taxonomy.tsv \
# --sample-metadata-fp $metadatapath \
# --sc-separated taxonomy

In [26]:
# !biom convert \
# -i exported/feature-table-tax.biom \
# -o exported/feature-table.tsv \
# --to-tsv

> process by decontam_mcm_its.R # May172020 chose not to use decontam

In [ ]:
# !qiime feature-table filter-features \
# --i-table feature-tables/table_mcm_its.qza \
# --m-metadata-file exported/contam_by_freq.txt \
# --p-exclude-ids True \
# --o-filtered-table exported/decontam_filtered_table.qza

In [29]:
# remove controls
# !qiime feature-table filter-samples \
# --i-table exported/decontam_filtered_table.qza \
# --m-metadata-file $metadatapath \
# --p-where "cul_type='Control'" \
# --p-exclude-ids \
# --o-filtered-table feature-tables/filtered_table_nocontrols.qza

!qiime feature-table filter-samples \
--i-table feature-tables/table_mcm_its.qza \
--m-metadata-file $metadatapath \
--p-where "cul_type='Control'" \
--p-exclude-ids \
--o-filtered-table feature-tables/filtered_table_nocontrols.qza

Saved FeatureTable[Frequency] to: feature-tables/filtered_table_nocontrols.qza


In [30]:
# get controls
!qiime feature-table filter-samples \
--i-table feature-tables/table_mcm_its.qza \
--m-metadata-file $metadatapath \
--p-where "cul_type='Control'" \
--o-filtered-table feature-tables/feature_table_controls.qza

Saved FeatureTable[Frequency] to: feature-tables/feature_table_controls.qza


## export to R analysis

In [5]:
!qiime tools export \
--input-path feature-tables/filtered_table_nocontrols.qza \
--output-path R_process/

!qiime tools export \
--input-path taxonomy/taxonomy_mcm_its.qza \
--output-path R_process/

Exported feature-tables/filtered_table_nocontrols.qza as BIOMV210DirFmt to directory R_process/
Exported taxonomy/taxonomy_mcm_its.qza as TSVTaxonomyDirectoryFormat to directory R_process/


In [6]:
!cp R_process/taxonomy.tsv R_process/biom-taxonomy.tsv

In [7]:
# change header
!sed -i '1 s/Feature ID/#OTUID/g; s/Taxon/taxonomy/g; s/Confidence/confidence/g' R_process/biom-taxonomy.tsv

In [8]:
!biom add-metadata \
-i R_process/feature-table.biom \
-o R_process/feature-table-tax.biom \
--observation-metadata-fp R_process/biom-taxonomy.tsv \
--sample-metadata-fp $metadatapath \
--sc-separated taxonomy

In [9]:
!biom convert \
-i R_process/feature-table-tax.biom \
-o R_process/feature-table.tsv \
--to-tsv

In [36]:
!qiime tools export \
--input-path rooted_tree.qza \
--output-path R_process/

Exported rooted_tree.qza as NewickDirectoryFormat to directory R_process/


## alpha rarefaction curves

In [40]:
!qiime diversity alpha-rarefaction \
--i-table feature-tables/table_mcm_its.qza \
--i-phylogeny rooted_tree.qza \
--m-metadata-file $metadatapath \
--p-steps 100 \
--p-max-depth 15000 \
--o-visualization alpha-rarecurve

Saved Visualization to: alpha-rarecurve.qzv


*END* move to R